In [ ]:
#Updating focus of ML with new combinations/optimizations. Uses sets of data from previous runs as well
#Things to look at:
##############
###### Using the random forest -
#explore optimizing landscape used for modeling - 1 part of the original library, 2, 1 of each original and terpolymer, etc - explore on one model, then optimize parameters on full model and minimum landscape
# explore how much data is used to train models (fractions of original and terpolymer more combos) 2d heat map of accuracy vs data - cross validation & for best performing
#optimize a best performance model using min landscape and min data

###### For both RF and Linear models
#look at the confusion matrix of the best performing models - split by terpolymer and bipolymers for each category

In [1]:
import pickle

#author: kfransen
#modeling of terpolymer library data with optimized models and model structures from the original library


#optimized library selections:
#list of [num estimators, max depth, max leaf nodes]


#original_rf selected for cross_val>0.7 and test score >0.72
original_rf=[[8,4,8],[16,16,128],[16,128,512],[32,256,8]]
#reduced_rf selected for cross_val>0.81
reduced_rf=[[8,2,8],[16,4,128],[64,256,8],[256,4,256]]
#original_SDG selected for maximum cross val/test score for each type of loss
original_SDG=[['hinge',0.1],['log_loss',0.01],['perceptron',0.001]]
#reduced SDG selected for maximum cross val/test score for each type of loss
reduced_SDG=[['hinge',0.1],['log_loss',0.1],['perceptron',0.0001]]

original_rf_max=[16,16,128]
reduced_rf_max=[64,256,8]




In [2]:
#import block

#imports
import numpy as np
import matplotlib.pyplot as plt
import shutil
import sys
import os.path
import json
import seaborn as sns
import pandas as pd
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys
from rdkit.Chem.AtomPairs import Pairs
from rdkit.Chem.rdFingerprintGenerator import GetRDKitFPGenerator
import ot
import csv
from pickle import dump
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import pairwise_distances
from scipy.optimize import minimize
from scipy.stats import bernoulli
from scipy.special import expit as sigmoid
from sklearn.datasets import make_moons
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import ConstantKernel, RBF, RationalQuadratic
from multiprocessing import Pool

#import EMD calculation from Jiale's project
from Polymer_Similarity import EMD_Calculation_pot as EMD

In [92]:
# function definition block
def str_to_list(string):
    #string='"'+str(string)+'"'
    counter=string.count(',')
    #print(counter)
    list=[]
    for i in range(counter):
        if counter ==1:
            string=string[1:len(string)-1]
        if i==0 and counter>1:
            j=string.index(',')
            list+=[string[2:j-1]]
            string=string[j+1:len(string)-1]
            if string[0]==' ':
                string=string[1:]
        elif i==counter-1:
            j=string.index(',')
            list+=[string[1:j-1]]
            if string[j+1]==' ':
                list+=[string[j+3:len(string)-1]]
            else:
                list+=[string[j+2:len(string)-1]]
        else:
            j=string.index(',')
            list+=[string[1:j-1]]
            if string[0]==' ':
                string=string[j+2:]
            else:
                string=string[j+1:]
        #print(string)

    return list

def concatenate_indices(indices,leave_out):
    #indices is a list of the split arrays, leave out is the index number of the test set
    newlist=[]
    test=None
    for i in range(len(indices)):
        if i==leave_out:
            test=indices[i]
        else:
            newlist+=[indices[i]]
    train=newlist[0]
    for i in range(1,len(newlist)):
        train=np.concatenate((train,newlist[i]),0)
    return [train,test]

def rf_cross_val(C, xs, ys,testx,testy,name):
    #C should be [n_estimators, max_depth,max_leaf_nnodes]
    #xs and ys give a list of the different dataset splices - must be same length, indices in each must match
        #this should be the full training set (will iterate which set is being used as the validation set for cross validation
    #added in option for testx and test y to be list of tests rather than just one test
    scores = []
    for i in range(0, len(xs)):
        X = concatenate_indices(xs, i)
        Y = concatenate_indices(ys, i)
        model = RandomForestClassifier(n_estimators=C[0], max_depth=C[1], min_samples_split=2, min_samples_leaf=1,max_leaf_nodes=C[2],random_state=42)
        model.fit(X[0], Y[0])
        scores += [model.score(X[1], Y[1])]
    score_array = np.array(scores)
    mean = np.mean(score_array)
    std = np.std(score_array)

    #now evaluate against unseen testing data
    X = concatenate_indices(xs, None)
    Y = concatenate_indices(ys, None)
    model = RandomForestClassifier(n_estimators=C[0], max_depth=C[1], min_samples_split=2, min_samples_leaf=1,max_leaf_nodes=C[2],random_state=42)
    model.fit(X[0], Y[0])
    if isinstance(testx,list):
        eval=[]
        for i in range(len(testx)):
            eval+=[model.score(testx[i],testy[i])]
    else:
        eval=model.score(testx,testy)
    with open("./Updated_Modeling/"+str(name)+"RF_[n_estim,max_depth,max_leaf]_"+str(C)+".pkl", "wb") as f:
        dump(model, f, protocol=5)
    return [mean, std, eval]

def train_rf(C,xs,ys,name):
    #C should be [n_estimators, max_depth,max_leaf_nnodes]
    #xs and ys should be list of the training data (no validation set, test set should be left out)
    # X = concatenate_indices(xs, None)
    # Y = concatenate_indices(ys, None)
    X=xs
    print('X',np.shape(X))
    # print('Y',np.shape(Y))
    Y=ys
    print('Y',np.shape(Y))
    model = RandomForestClassifier(n_estimators=C[0], max_depth=C[1], min_samples_split=2, min_samples_leaf=1, max_leaf_nodes=C[2],random_state=42)
    model.fit(X, Y)
    with open("./Updated_Modeling/"+str(name)+"RF_[n_estim,max_depth,max_leaf]_"+str(C)+".pkl", "wb") as f:
        dump(model, f, protocol=5)
    return model

def update_rf(old_model,C,xs,ys):
    #this takes an existing trained rf and fits additional trees given additional training data
    #C should be the number of additional estimators (trees) to fit
    #xs and ys should be list of the training data (no validation set, test set should be left out)
    n_estims=len(old_model.estimators_)

    X = concatenate_indices(xs, None)
    Y = concatenate_indices(ys, None)
    model_list=[]
    estimator_amounts=[]
    model = old_model.set_params(n_estimators=C+n_estims,warm_start=True)
    model.fit(X[0], Y[0])
    return model


def lin_cross_val(C, xs, ys,xtest,ytest,name):
    #C should be [loss, alpha]
    #xs and ys give a list of the different dataset splices - must be same length, indices in each must match
        #this should be the full training set (will iterate which set is being used as the validation set for cross validation
    scores = []
    for i in range(0, len(xs)):
        X = concatenate_indices(xs, i)
        Y = concatenate_indices(ys, i)
        model =SGDClassifier(loss=C[0], alpha=C[1],shuffle=True)
        model.fit(X[0], Y[0])
        scores += [model.score(X[1], Y[1])]
    score_array = np.array(scores)
    mean = np.mean(score_array)
    std = np.std(score_array)
    X = concatenate_indices(xs, None)
    Y = concatenate_indices(ys, None)
    model = SGDClassifier(loss=C[0], alpha=C[1], shuffle=True)
    model.fit(X[0],Y[0])
    eval=model.score(xtest,ytest)
    #save the model
    with open("./240527_with_Terpolymers_Optimization Models/SGD_"+str(name)+"_[loss,alpha]_"+str(C)+".pkl", "wb") as f:
        dump(model, f, protocol=5)
    return [mean, std,eval]

def train_lin(C, xs,ys):
    X = concatenate_indices(xs, None)
    Y = concatenate_indices(ys, None)
    model = SGDClassifier(loss=C[0], alpha=C[1], shuffle=True,random_state=42)
    model.fit(X[0], Y[0])
    return model

def update_lin(old_model, xs, ys):
    #xs are additional new data that needs to be fit
    #ys are labels corresponding to xs
    class_biodeg=[0,1] #pretty sure since we use 0 or 1 this is all we've got
    model=old_model.partial_fit(xs,ys,classes=class_biodeg)
    return model

def get_EMD_array(sets,basis,EMD_values=["RDKFingerprint","Tanimoto",5,512]):
    #sets = list of sets (pd dataframes) that need EMD calculations against the basis
    #if basis=None then the concatenation of the sets is used as basis. If not, give polymer basis
    #EMD_values gives [embedding_function, similarity_score_function, radius, num_bits]

    EMD_arrays=[]
    for k in range(len(sets)):
        col = []
        for i in range(len(sets[k].index)):
            row = []
            for j in range(len(basis.index)):
                X = sets[k]['SMILES_List'][i]
                #print(X)
                if isinstance(X,str):
                    X = str_to_list(X)
                #print(X)
                Y = sets[k]['Ratio_List'][i]
                #print(basis['SMILES_List'])
                #print("basis",basis['SMILES_List'][j])
                Z = basis['SMILES_List'][j]
                #print(Z)
                if isinstance(Z,str):
                    Z = str_to_list(Z)
                #print(Z)
                W = basis['Ratio_List'][j]
                # W=str_to_list(W)
                #print(X, Y, Z, W)
                score = EMD(
                    query_smiles_list=X,
                    query_smiles_weight_list=Y,
                    target_smiles_list=Z,
                    target_smiles_weight_list=W,
                    embedding_function=EMD_values[0],
                    similarity_score_function=EMD_values[1],
                    radius=EMD_values[2],
                    num_bits=EMD_values[3],
                )
                row += [score]
            col += [row]
        # save the list of lists as a npy array
        formed = np.array(col)
        EMD_arrays+=[formed]
    return EMD_arrays


In [4]:
def get_smiles_and_ratios_list(dataframe):
    names=dataframe['Polymer_Name_P'].to_list()
    ratio1=dataframe['Ratio 1'].to_list()
    ratio2=dataframe['Ratio 2'].to_list()
    ratio3=dataframe['Ratio 3'].to_list()
    smiles=[]
    ratios=[]
    monomers=pd.read_csv('./Monomer Info_DMTDMI.csv',header=0)
    for i in range(len(names)):
        name=names[i]
        smile=[]
        smile+=[str(monomers.loc[monomers['Polymer Name Abbr.']==name[0:3],'SMILES_Monomers'].item())]
        smile+=[str(monomers.loc[monomers['Polymer Name Abbr.']==name[6:9],'SMILES_Monomers'].item())]
        smile+=[str(monomers.loc[monomers['Polymer Name Abbr.']==name[12:15],'SMILES_Monomers'].item())]
        #print(smile)
        smiles+=[smile]
        #print(smiles)
        ratios+=[[ratio1[i]/2,ratio2[i]/2,ratio3[i]/2]]
    return[smiles,ratios]

In [5]:
#make EMD matrix of ter polymer test set (set 5) against original training set and reduced training set
ter_test=pd.read_csv('./Terpolymer_Splits/split_5.csv',header=0)
test_ter_list=get_smiles_and_ratios_list(ter_test)


In [13]:
def get_EMD_array_lists(sets,ratios,basis,EMD_values=["RDKFingerprint","Tanimoto",5,512]):
    #sets = list of sets (each set is a list of smiles lists dataframes) that need EMD calculations against the basis
    #ratios is a list of ratio lists corresponding to the sets
    #if basis=None then the concatenation of the sets is used as basis. If not, give polymer basis
    #EMD_values gives [embedding_function, similarity_score_function, radius, num_bits]

    EMD_arrays=[]
    for k in range(len(sets)):
        col = []
        for i in range(len(sets[k])):
            row = []
            for j in range(len(basis.index)):
                X = sets[k][i]
                print(X)
                Y = ratios[k][i]
                #print(basis['SMILES_List'])
                #print("basis",basis['SMILES_List'][j])
                Z = basis['SMILES_List'][j]
                #print(Z)
                Z = str_to_list(Z)
                W = basis['Ratio_List'][j]
                # W=str_to_list(W)
                #print(X, Y, Z, W)
                score = EMD(
                    query_smiles_list=X,
                    query_smiles_weight_list=Y,
                    target_smiles_list=Z,
                    target_smiles_weight_list=W,
                    embedding_function=EMD_values[0],
                    similarity_score_function=EMD_values[1],
                    radius=EMD_values[2],
                    num_bits=EMD_values[3],
                )
                row += [score]
            col += [row]
        # save the list of lists as a npy array
        formed = np.array(col)
        EMD_arrays+=[formed]
    return EMD_arrays

In [15]:
#load original and reduced data
os0=pd.read_csv('./ML_splits/original_split0.csv',header=0)
os1=pd.read_csv('./ML_splits/original_split1.csv',header=0)
os2=pd.read_csv('./ML_splits/original_split2.csv',header=0)
os3=pd.read_csv('./ML_splits/original_split3.csv',header=0)
os4=pd.read_csv('./ML_splits/original_split4.csv',header=0)
os5=pd.read_csv('./ML_splits/original_split5.csv',header=0)
y_set0=np.load('./ML_splits/Biodeg_fororiginal_split0_vsfull_library.npy')
y_set1=np.load('./ML_splits/Biodeg_fororiginal_split1_vsfull_library.npy')
y_set2=np.load('./ML_splits/Biodeg_fororiginal_split2_vsfull_library.npy')
y_set3=np.load('./ML_splits/Biodeg_fororiginal_split3_vsfull_library.npy')
y_set4=np.load('./ML_splits/Biodeg_fororiginal_split4_vsfull_library.npy')
y_set5=np.load('./ML_splits/Biodeg_fororiginal_split5_vsfull_library.npy')
os_train=[os0,os1,os2,os3,os4] #the training data for the cross validation/full model
os_train_pd=pd.concat(os_train).reset_index()
rl0=pd.read_csv('./Reduced_ML_Splits/reduced_split0.csv',header=0)
rl1=pd.read_csv('./Reduced_ML_Splits/reduced_split1.csv',header=0)
rl2=pd.read_csv('./Reduced_ML_Splits/reduced_split2.csv',header=0)
rl3=pd.read_csv('./Reduced_ML_Splits/reduced_split3.csv',header=0)
rl4=pd.read_csv('./Reduced_ML_Splits/reduced_split4.csv',header=0)
rl5=pd.read_csv('./Reduced_ML_Splits/reduced_split5.csv',header=0)
rl_train=[rl0,rl1,rl2,rl3,rl4] #the training data used for cross validation/full model
rl_train_pd=pd.concat(rl_train).reset_index()


In [8]:
#get the EMD arrays for terpolymer testing vs original and reduced libraries
tertest_emd_vs_orig=get_EMD_array_lists([test_ter_list[0]],[test_ter_list[1]],os_train_pd)
tertest_emd_vs_reduc=get_EMD_array_lists([test_ter_list[0]],[test_ter_list[1]],rl_train_pd)
np.save('./Terpolymer_Splits/EMD_split'+str(5)+'_vs_orig',tertest_emd_vs_orig,allow_pickle=False)
np.save('./Terpolymer_Splits/EMD_split'+str(5)+'_vs_reduc',tertest_emd_vs_reduc,allow_pickle=False)

['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(cc

In [58]:
#load all the models of interest to test against and record performance vs ter_polymer test set
import pickle
Xval=np.load('./Terpolymer_Splits/EMD_split5_vs_orig.npy')[0]
print(Xval)
Yval=np.load('./Terpolymer_Splits/split5_ys.npy')
for i in range(len(original_rf)):
    C=original_rf[i]
    with open("./240523_Optimization_Models/RF_os_estim_"+str(C[0])+"_depth_"+str(C[1])+"_leafnodes_"+str(C[2])+".pkl", "rb") as f:
        model=pickle.load(f)
    value=model.score(Xval,Yval)
    with open('optimal_model_evaluations.csv', 'a', newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow(['RF_Original',str(C),value])
Xval_r=np.load('./Terpolymer_Splits/EMD_split5_vs_reduc.npy')[0]

for i in range(len(reduced_rf)):
    C=reduced_rf[i]
    with open("./240523_Optimization_Models/RF_rl_estim_"+str(C[0])+"_depth_"+str(C[1])+"_leafnodes_"+str(C[2])+".pkl", "rb") as f:
        model=pickle.load(f)
    value=model.score(Xval_r,Yval)
    with open('optimal_model_evaluations.csv', 'a', newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow(['RF_Reduced',str(C),value])


[[0.57930624 0.62080046 0.87122345 ... 0.78859193 0.79883838 0.73437108]
 [0.37367443 0.63194855 0.82610905 ... 0.65741268 0.80231808 0.90195713]
 [0.53553107 0.76516824 0.82444558 ... 0.74303936 0.82571962 0.90526887]
 ...
 [0.53592317 0.54885791 0.82925429 ... 0.84916024 0.80970172 0.86302156]
 [0.4696234  0.61988574 0.86328483 ... 0.80776274 0.77171768 0.88482908]
 [0.45793267 0.62704258 0.87832992 ... 0.78080723 0.81512066 0.88990407]]


In [59]:
#original_SDG selected for
original_SDG=[['hinge',0.1],['log_loss',0.01],['perceptron',0.001]]
#reduced SDG selected for
reduced_SDG=[['hinge',0.1],['log_loss',0.1],['perceptron',0.0001]]
for i in range(len(original_SDG)):
    C=original_SDG[i]
    with open("./240523_Optimization_Models/SGD_os"+"_loss_"+str(C[0])+"_alpha_"+str(C[1])+".pkl", "rb") as f:
        model=pickle.load(f)
    value=model.score(Xval,Yval)
    with open('optimal_model_evaluations.csv', 'a', newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow(['SDG_Original',str(C),value])
Xval_r=np.load('./Terpolymer_Splits/EMD_split5_vs_reduc.npy')[0]

for i in range(len(reduced_SDG)):
    C=reduced_SDG[i]
    with open("./240523_Optimization_Models/SGD_rl"+"_loss_"+str(C[0])+"_alpha_"+str(C[1])+".pkl", "rb") as f:
        model=pickle.load(f)
    value=model.score(Xval_r,Yval)
    with open('optimal_model_evaluations.csv', 'a', newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow(['SDG_Reduced',str(C),value])

In [44]:
#generate all the possible EMD 'landscapes' that we may be interested in from the ter polymer splits + the original library and reduced library#load original and reduced data
os0=pd.read_csv('./ML_splits/original_split0.csv',header=0)
os1=pd.read_csv('./ML_splits/original_split1.csv',header=0)
os2=pd.read_csv('./ML_splits/original_split2.csv',header=0)
os3=pd.read_csv('./ML_splits/original_split3.csv',header=0)
os4=pd.read_csv('./ML_splits/original_split4.csv',header=0)
os5=pd.read_csv('./ML_splits/original_split5.csv',header=0)
y_set0=np.load('./ML_splits/Biodeg_fororiginal_split0_vsfull_library.npy')
y_set1=np.load('./ML_splits/Biodeg_fororiginal_split1_vsfull_library.npy')
y_set2=np.load('./ML_splits/Biodeg_fororiginal_split2_vsfull_library.npy')
y_set3=np.load('./ML_splits/Biodeg_fororiginal_split3_vsfull_library.npy')
y_set4=np.load('./ML_splits/Biodeg_fororiginal_split4_vsfull_library.npy')
y_set5=np.load('./ML_splits/Biodeg_fororiginal_split5_vsfull_library.npy')
os_train=[os0,os1,os2,os3,os4,os5] #the training data for the cross validation/full model

rl0=pd.read_csv('./Reduced_ML_Splits/reduced_split0.csv',header=0)
rl1=pd.read_csv('./Reduced_ML_Splits/reduced_split1.csv',header=0)
rl2=pd.read_csv('./Reduced_ML_Splits/reduced_split2.csv',header=0)
rl3=pd.read_csv('./Reduced_ML_Splits/reduced_split3.csv',header=0)
rl4=pd.read_csv('./Reduced_ML_Splits/reduced_split4.csv',header=0)
rl5=pd.read_csv('./Reduced_ML_Splits/reduced_split5.csv',header=0)
rl_train=[rl0,rl1,rl2,rl3,rl4,rl5] #the training data used for cross validation/full model
# print(rl_train)

os_train_xs=[]
os_train_ys=[]
for dataframe in os_train:
    os_train_xs+=[dataframe[['SMILES_List','Ratio_List']]]
    os_train_ys+=[dataframe['Biodegradability'].to_numpy()]
rl_train_xs=[]
rl_train_ys=[]
for dataframe in rl_train:
    rl_train_xs+=[dataframe[['SMILES_List','Ratio_List']]]
    rl_train_ys+=[dataframe['Biodegradability'].to_numpy()]
#different landscape sets to look at
original_only_incr=[] #list of the pd dataframes with only original library members as landscape
reduced_only_incr=[] #list of the pd dataframes with only reduced library members as landscape
original_ter_incr=[] #list of pd dataframes with original and terpolymers as landscape in increasing amounts
reduced_ter_incr=[] #list of pd dataframes with reduced and terpolymers as landscape in increasing amounts

#list of np arrays that correspond to list of pd dataframes for training the models
original_only_ys=[]
reduced_only_ys=[]
original_ter_ys=[]
reduced_ter_ys=[]

original_only=os_train_xs[0]
reduced_only=rl_train_xs[0]
original_total=os_train_xs[0]
reduced_total=rl_train_xs[0]
original_only_y=os_train_ys[0]
reduced_only_y=rl_train_ys[0]
original_ter_y=os_train_ys[0]
reduced_ter_y=rl_train_ys[0]
# print(np.size(reduced_ter_y))
for i in range(5):
    #print(i)
    A_pd=pd.read_csv('./Terpolymer_Splits/split_'+str(i)+'_A.csv',header=0)
    B_pd=pd.read_csv('./Terpolymer_Splits/split_'+str(i)+'_B.csv',header=0)
    A_np=np.load('./Terpolymer_Splits/split'+str(i)+'_A_ys.npy')
    B_np=np.load('./Terpolymer_Splits/split'+str(i)+'_B_ys.npy')
    values_A=get_smiles_and_ratios_list(A_pd)
    #print(values_A[0])
    values_B=get_smiles_and_ratios_list(B_pd)
    #print(values_B)
    A_newpd=pd.DataFrame({'SMILES_List':values_A[0],'Ratio_List':values_A[1]})
    #print(A_newpd)
    B_newpd=pd.DataFrame({'SMILES_List':values_B[0],'Ratio_List':values_B[1]})
    #print(original_total)
    if i ==0:
        original_total=pd.concat((os_train_xs[i],A_newpd,B_newpd)).reset_index(drop=True)
        #print(original_total)
        reduced_total=pd.concat([rl_train_xs[i],A_newpd,B_newpd]).reset_index(drop=True)
        original_ter_y=np.concatenate([os_train_ys[i],A_np,B_np])
        reduced_ter_y=np.concatenate([rl_train_ys[i],A_np,B_np])
        # print(np.size(reduced_ter_y))
        print('original', str(i),np.size(original_ter_y))
        print('reduced', str(i),np.size(reduced_ter_y))
    else:
        original_only=pd.concat([original_only,os_train_xs[i]]).reset_index(drop=True)
        original_total=pd.concat((original_total,os_train_xs[i],A_newpd,B_newpd)).reset_index(drop=True)
        original_only_y=np.concatenate([original_only_y,os_train_ys[i]])
        original_ter_y=np.concatenate([original_ter_y,os_train_ys[i],A_np,B_np])

        reduced_only=pd.concat([reduced_only,rl_train_xs[i]]).reset_index(drop=True)
        reduced_total=pd.concat([reduced_total,rl_train_xs[i],A_newpd,B_newpd]).reset_index(drop=True)
        reduced_only_y=np.concatenate([reduced_only_y,rl_train_ys[i]])
        reduced_ter_y=np.concatenate([reduced_ter_y,rl_train_ys[i],A_np,B_np])
        # print(np.size(reduced_ter_y))
        print('original', str(i),np.size(original_ter_y))
        print('reduced', str(i),np.size(reduced_ter_y))
    original_only_incr+=[original_only]
    original_only_ys+=[original_only_y]
    original_ter_incr+=[original_total]
    original_ter_ys+=[original_ter_y]
    reduced_only_incr+=[reduced_only]
    reduced_only_ys+=[reduced_only_y]
    reduced_ter_incr+=[reduced_total]
    reduced_ter_ys+=[reduced_ter_y]
# print(np.size(reduced_ter_ys[-1]))

with open('./Updated_Modeling/original_only_incr_landscape.pkl',"wb") as f:
    dump(original_only_incr,f,protocol=5)
with open('./Updated_Modeling/original_only_incr_landscape_ys.pkl',"wb") as f:
    dump(original_only_ys,f,protocol=5)
with open('./Updated_Modeling/original_ter_incr_landscape.pkl',"wb") as f:
    dump(original_ter_incr,f,protocol=5)
with open('./Updated_Modeling/original_ter_incr_landscape_ys.pkl',"wb") as f:
    dump(original_ter_ys,f,protocol=5)

with open('./Updated_Modeling/reduced_only_incr_landscape.pkl',"wb") as f:
    dump(reduced_only_incr,f,protocol=5)
with open('./Updated_Modeling/reduced_only_incr_landscape_ys.pkl',"wb") as f:
    dump(reduced_only_ys,f,protocol=5)
with open('./Updated_Modeling/reduced_ter_incr_landscape.pkl',"wb") as f:
    dump(reduced_ter_incr,f,protocol=5)
with open('./Updated_Modeling/reduced_ter_incr_landscape_ys.pkl',"wb") as f:
    dump(reduced_ter_ys,f,protocol=5)


original 0 157
reduced 0 87
original 1 315
reduced 1 175
original 2 474
reduced 2 265
original 3 638
reduced 3 359
original 4 809
reduced 4 460


In [42]:
#dataframe os5 is the original library test
#dataframe rl5 is the reduced library test

#make the 3 different test set evaluations
original_tests=os5[['SMILES_List','Ratio_List']]
original_tests_ys_np=os5['Biodegradability'].to_numpy()
reduced_tests=rl5[['SMILES_List','Ratio_List']]
reduced_tests_ys_np=rl5['Biodegradability'].to_numpy()

ter_test_pd=pd.DataFrame({'SMILES_List':test_ter_list[0],'Ratio_List':test_ter_list[1]})
Yval=np.load('./Terpolymer_Splits/split5_ys.npy')

original_test_pds=[original_tests,ter_test_pd,pd.concat([original_tests,ter_test_pd]).reset_index(drop=True)]
original_test_ylist=[original_tests_ys_np,Yval,np.concatenate([original_tests_ys_np,Yval])]
reduced_test_pds=[reduced_tests,ter_test_pd,pd.concat([reduced_tests,ter_test_pd]).reset_index(drop=True)]
reduced_test_ylist=[reduced_tests_ys_np,Yval,np.concatenate([reduced_tests_ys_np,Yval])]

orig_train=original_ter_incr[-1]
Y_orig_train=original_ter_ys[-1]
reduced_train=reduced_ter_incr[-1]
Y_red_train=reduced_ter_ys[-1]


In [21]:
#get the training and test X arrays (we got the y arrays above)
#the dataset we want to use is either all the training data from the original with the terpolymers or from the reduced with all the terpolymers
orig_train=original_ter_incr[-1]
Y_orig_train=original_ter_ys[-1]
reduced_train=reduced_ter_incr[-1]
Y_red_train=reduced_ter_ys[-1]
#Now calculate the EMD matrices for our 4 landscape sets
#no terpolymer info in landscapes, original library
original_only_emd_train=[] #this should end up being a list of training arrays
original_only_emd_test=[] #this ends up being a list of list since for each basis there is a list of test sets
#terpolymers in the landscapes, original library
original_ter_emd_train=[] #this should end up being a list of training arrays
original_ter_emd_test=[] #this ends up being a list of list since for each basis there is a list of test sets
#no terpolymer info in landscape, reduced library
reduced_only_emd_train=[] #this should end up being a list of training arrays
reduced_only_emd_test=[] #this ends up being a list of list since for each basis there is a list of test sets
#terpolymer in the landscape, reduced library
reduced_ter_emd_train=[] #this should end up being a list of training arrays
reduced_ter_emd_test=[] #this ends up being a list of list since for each basis there is a list of test sets
for b in original_only_incr:
    print(b)
    array=get_EMD_array([orig_train],b)
    original_only_emd_train+=[array[0]]
    array=get_EMD_array(original_test_pds,b)
    original_only_emd_test+=[array]
#dump all the data into pickles
with open('./Updated_Modeling/original_only_landscapes_train_emd_list.pkl',"wb") as f:
    dump(original_only_emd_train,f,protocol=5)
with open('./Updated_Modeling/original_only_landscapes_test_emd_list.pkl',"wb") as f:
    dump(original_only_emd_test,f,protocol=5)
with open('./Updated_Modeling/original_testys_np_list.pkl',"wb") as f:
    dump(original_test_ylist,f,protocol=5)
for b in original_ter_incr:
    print(b)
    array=get_EMD_array([orig_train],b)
    original_ter_emd_train+=[array[0]]
    array=get_EMD_array(original_test_pds,b)
    original_ter_emd_test+=[array]
#dump all the data into pickles
with open('./Updated_Modeling/original_ter_landscapes_train_emd_list.pkl',"wb") as f:
    dump(original_ter_emd_train,f,protocol=5)
with open('./Updated_Modeling/original_ter_landscapes_test_emd_list.pkl',"wb") as f:
    dump(original_ter_emd_test,f,protocol=5)
#############again for reduced library
for b in reduced_only_incr:
    print(b)
    array=get_EMD_array([reduced_train],b)
    reduced_only_emd_train+=[array[0]]
    array=get_EMD_array(reduced_test_pds,b)
    reduced_only_emd_test+=[array]
#dump all the data into pickles
with open('./Updated_Modeling/reduced_only_landscapes_train_emd_list.pkl',"wb") as f:
    dump(reduced_only_emd_train,f,protocol=5)
with open('./Updated_Modeling/reduced_only_landscapes_test_emd_list.pkl',"wb") as f:
    dump(reduced_only_emd_test,f,protocol=5)
with open('./Updated_Modeling/reduced_testys_np_list.pkl',"wb") as f:
    dump(reduced_test_ylist,f,protocol=5)
for b in reduced_ter_incr:
    print(b)
    array=get_EMD_array([reduced_train],b)
    reduced_ter_emd_train+=[array[0]]
    array=get_EMD_array(reduced_test_pds,b)
    reduced_ter_emd_test+=[array]
#dump all the data into pickles
with open('./Updated_Modeling/reduced_ter_landscapes_train_emd_list.pkl',"wb") as f:
    dump(reduced_ter_emd_train,f,protocol=5)
with open('./Updated_Modeling/reduced_ter_landscapes_test_emd_list.pkl',"wb") as f:
    dump(reduced_ter_emd_test,f,protocol=5)

                                           SMILES_List Ratio_List
0               ['C(=O)c(cc1)ccc1C(=O)', 'OCCCCCCCCO']     [1, 1]
1    ['C(=O)c(cc1)ccc1C(=O)', 'Oc1c(C)cc(cc1(C))C(C...     [1, 1]
2    ['C(=O)C(=O)', 'Oc1ccc(cc1)C(C)(c2ccccc2)c3ccc...     [1, 1]
3                           ['C(=O)CC(=O)', 'OCC=CCO']     [1, 1]
4                       ['C(=O)C(=O)', 'OCCCCCCCCCCO']     [1, 1]
..                                                 ...        ...
104         ['C(=O)COC(=O)CO', 'C(=O)CCCCCCCCCCCCCCO']     [2, 1]
105          ['C(=O)COC(=O)CO', 'C(=O)C=Cc(cccc1)c1O']     [2, 1]
106       ['C(=O)C=Cc(cccc1)c1O', 'C(=O)CCCCC(CCCC)O']     [2, 1]
107      ['C(=O)C=Cc(cccc1)c1O', 'C(=O)C=CCC(CCCCC)O']     [2, 1]
108                     ['C(=O)COC(=O)CO', 'C(=O)CCO']     [2, 1]

[109 rows x 2 columns]
                                           SMILES_List Ratio_List
0               ['C(=O)c(cc1)ccc1C(=O)', 'OCCCCCCCCO']     [1, 1]
1    ['C(=O)c(cc1)ccc1C(=O)', 'Oc1c(C)cc(cc1(C))C(C.

In [43]:
#retraining max performance RF models with different landscape ranges and evaluating them
original_rf_max=[16,16,128]
reduced_rf_max=[64,256,8]

with open('./Updated_Modeling/Landscape_study.csv','w',newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow(['Landscape Category','Landscape Number','RF Parameters [max trees, max depth, max leaf nodes]','Binary Polymers Score','Terpolymer Test Score','Combined Test Score'])
#original only
for i in range(len(original_only_emd_train)):
    #first open a file to record all the data
    C=original_rf_max
    #train the model with the data and specific landscape
    model=train_rf(C,[original_only_emd_train[i]],[Y_orig_train])
    #score the original model with the testing parameters of interest
    Xs=original_only_emd_test[i]
    Ys=original_test_ylist
    orig_score=model.score(Xs[0],Ys[0])
    ter_score=model.score(Xs[1],Ys[1])
    combined_score=model.score(Xs[2],Ys[2])

    with open('./Updated_Modeling/Landscape_study.csv','a',newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow(['Original Only',i,str(C),orig_score,ter_score,combined_score])
#original with terpolymers
for i in range(len(original_ter_emd_train)):
    #first open a file to record all the data
    C=original_rf_max
    #train the model with the data and specific landscape
    model=train_rf(C,[original_ter_emd_train[i]],[Y_orig_train])
    #score the original model with the testing parameters of interest
    Xs=original_ter_emd_test[i]
    Ys=original_test_ylist
    orig_score=model.score(Xs[0],Ys[0])
    ter_score=model.score(Xs[1],Ys[1])
    combined_score=model.score(Xs[2],Ys[2])

    with open('./Updated_Modeling/Landscape_study.csv','a',newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow(['Original and Ter',i,str(C),orig_score,ter_score,combined_score])
#reduced only
for i in range(len(reduced_only_emd_train)):
    #first open a file to record all the data
    C=reduced_rf_max
    #train the model with the data and specific landscape
    print(np.shape(reduced_only_emd_train[i]),np.shape(Y_red_train))
    model=train_rf(C,[reduced_only_emd_train[i]],[Y_red_train])
    #score the original model with the testing parameters of interest
    Xs=reduced_only_emd_test[i]
    Ys=reduced_test_ylist
    orig_score=model.score(Xs[0],Ys[0])
    ter_score=model.score(Xs[1],Ys[1])
    combined_score=model.score(Xs[2],Ys[2])

    with open('./Updated_Modeling/Landscape_study.csv','a',newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow(['Reduced Only',i,str(C),orig_score,ter_score,combined_score])
#reduced with terpolymers
for i in range(len(reduced_ter_emd_train)):
    #first open a file to record all the data
    C=reduced_rf_max
    #train the model with the data and specific landscape
    model=train_rf(C,[reduced_ter_emd_train[i]],[Y_red_train])
    #score the original model with the testing parameters of interest
    Xs=reduced_ter_emd_test[i]
    Ys=reduced_test_ylist
    orig_score=model.score(Xs[0],Ys[0])
    ter_score=model.score(Xs[1],Ys[1])
    combined_score=model.score(Xs[2],Ys[2])

    with open('./Updated_Modeling/Landscape_study.csv','a',newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow(['Reduced and Ter',i,str(C),orig_score,ter_score,combined_score])


(460, 39) (460,)
(460, 78) (460,)
(460, 118) (460,)
(460, 157) (460,)
(460, 197) (460,)


In [50]:
#minimum landscape seems to be fine for use for representation of the library, interestingly not using terpolymers seems to perform better than using them as part of the representation

number_estimators = [2**0, 2**1, 2**2, 2**3, 2**4, 2**5, 2**6, 2**7, 2**8, 2**9, 2**10]
max_depth = [1,2,4,8,16,32,64,128,256]
min_samples_split = [2]
min_samples_leaf = [1]
max_leaf_nodes = [2,4,8,16,32,64,128,256,512]
#make a list of all the combinations of parameters to run
combos=[]
for i in number_estimators:
    for j in max_depth:
        for k in max_leaf_nodes:
            combos+=[[i,j,k]]


#sizes of training subsets for cross val:
# #original 0 157
# reduced 0 87
# original 1 315
# reduced 1 175
# original 2 474
# reduced 2 265
# original 3 638
# reduced 3 359
# original 4 809
# reduced 4 460
#reduced original only basis
reduced_only_train_emd_0=reduced_only_emd_train[0][0:87,:]
Y_red_train_0=Y_red_train[0:87]
reduced_only_train_emd_1=reduced_only_emd_train[0][87:175,:]
Y_red_train_1=Y_red_train[87:175]
reduced_only_train_emd_2=reduced_only_emd_train[0][175:265,:]
Y_red_train_2=Y_red_train[175:265]
reduced_only_train_emd_3=reduced_only_emd_train[0][265:359,:]
Y_red_train_3=Y_red_train[265:359]
reduced_only_train_emd_4=reduced_only_emd_train[0][359:460,:]
Y_red_train_4=Y_red_train[359:460]
#original only basis
original_only_train_emd_0=original_only_emd_train[0][0:157,:]
Y_orig_train_0=Y_orig_train[0:157]
original_only_train_emd_1=original_only_emd_train[0][157:315,:]
Y_orig_train_1=Y_orig_train[157:315]
original_only_train_emd_2=original_only_emd_train[0][315:474,:]
Y_orig_train_2=Y_orig_train[315:474]
original_only_train_emd_3=original_only_emd_train[0][474:638,:]
Y_orig_train_3=Y_orig_train[474:638]
original_only_train_emd_4=original_only_emd_train[0][638:809,:]
Y_orig_train_4=Y_orig_train[638:809]

#split up training data for cross val optimization
reduced_only_training_sets=[reduced_only_train_emd_0,reduced_only_train_emd_1,reduced_only_train_emd_2,reduced_only_train_emd_3,reduced_only_train_emd_4]
y_reduced_only_sets=[Y_red_train_0,Y_red_train_1,Y_red_train_2,Y_red_train_3,Y_red_train_4]
original_only_training_sets=[original_only_train_emd_0,original_only_train_emd_1,original_only_train_emd_2,original_only_train_emd_3,original_only_train_emd_4]
y_original_only_sets=[Y_orig_train_0,Y_orig_train_1,Y_orig_train_2,Y_orig_train_3,Y_orig_train_4]

#test sets for optimization
Xs_red=reduced_only_emd_test[0]
Ys_red=reduced_test_ylist
Xs_orig=original_only_emd_test[0]
Ys_orig=original_test_ylist

with open('./Updated_Modeling/optimization_with_all_data.csv', 'w', newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow(['Dataset','Number of Estimators','Max Depth','Min Samples Split','Min Samples Leaf','Max Leaf Nodes','Mean Cross Val Score','STD','Eval [Library, Ter, Combined]'])
for C in combos:
    output=rf_cross_val(C,reduced_only_training_sets,y_reduced_only_sets,Xs_red,Ys_red,'reduced')
    with open('./Updated_Modeling/optimization_with_all_data.csv', 'a', newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow(['Reduced',C[0],C[1],2,1,C[2],output[0],output[1],output[2]])
    output=rf_cross_val(C,original_only_training_sets,y_original_only_sets,Xs_orig,Ys_orig,'original')
    with open('./Updated_Modeling/optimization_with_all_data.csv', 'a', newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow(['Original',C[0],C[1],2,1,C[2],output[0],output[1],output[2]])

In [101]:
#take the optimized models with the minimum representations to study how different amounts of training data impact the accuracy

#need to construct the matrix of training data
#we need list of the emd matrixes for each original set vs original, reduced set vs reduced and terpolymer sets vs each. plus corresponding ys
orig_optim_C=[32,32,64]
reduced_optim_C=[64,8,64]
#[0, 25, 23, 25, 24, 25, 25, 28, 27, 30, 31]
#split up training data for cross val optimization
reduced_only_training_sets=[reduced_only_train_emd_0[:-48],reduced_only_train_emd_1[:-49],reduced_only_train_emd_2[:-50],reduced_only_train_emd_3[:-55],reduced_only_train_emd_4[:-61]]
y_reduced_only_sets=[Y_red_train_0[:-48],Y_red_train_1[:-49],Y_red_train_2[:-50],Y_red_train_3[:-55],Y_red_train_4[:-61]]
original_only_training_sets=[original_only_train_emd_0[:-48],original_only_train_emd_1[:-49],original_only_train_emd_2[:-50],original_only_train_emd_3[:-55],original_only_train_emd_4[:-61]]
y_original_only_sets=[Y_orig_train_0[:-48],Y_orig_train_1[:-49],Y_orig_train_2[:-50],Y_orig_train_3[:-55],Y_orig_train_4[:-61]]
ter_training_set=[reduced_only_train_emd_0[-48:],reduced_only_train_emd_1[-49:],reduced_only_train_emd_2[-50:],reduced_only_train_emd_3[-55:],reduced_only_train_emd_4[-61:]]
y_ter_sets=[Y_red_train_0[-48:],Y_red_train_1[-49:],Y_red_train_2[-50:],Y_red_train_3[-55:],Y_red_train_4[-61:]]
terog_sets=[original_only_train_emd_0[-48:],original_only_train_emd_1[-49:],original_only_train_emd_2[-50:],original_only_train_emd_3[-55:],original_only_train_emd_4[-61:]]
y_terog_sets=[Y_orig_train_0[-48:],Y_orig_train_1[-49:],Y_orig_train_2[-50:],Y_orig_train_3[-55:],Y_orig_train_4[-61:]]

In [102]:
#train and evaluate with the different amounts of data from the training sets
original_data_eval_binary=[] #list of lists - each 'row' or list is a different amount of library data, each value in the row is different amount of terpolymer data
original_data_eval_ter=[]
original_data_eval_combo=[]
for i in range(len(original_only_training_sets)):
    print("i",i)
    sublist_binary=[]
    sublist_ter=[]
    sublist_combo=[]
    for j in range(len(terog_sets)):
        print("j",j)
        print(i,np.shape(original_only_training_sets[i]),j,np.shape(terog_sets[j]))
        xset=np.concatenate(original_only_training_sets[:i+1]+terog_sets[:j+1],axis=0)
        ys=np.concatenate(y_original_only_sets[:i+1]+y_terog_sets[:j+1],axis=0)
        # print(np.shape(original_only_training_sets[i]),np.shape(terog_sets[j]))
        trained=train_rf(orig_optim_C,xset,ys,'original_data_optim_og_sets_'+str(i)+'_ter_sets_'+str(j))
        eval=[]
        for k in range(len(Xs_orig)):
            eval+=[trained.score(Xs_orig[k],Ys_orig[k])]
        sublist_binary+=[eval[0]]
        sublist_ter+=[eval[1]]
        sublist_combo+=[eval[2]]
    original_data_eval_binary+=[sublist_binary]
    original_data_eval_ter+=[sublist_ter]
    original_data_eval_combo+=[sublist_combo]
numpy_binary=np.array(original_data_eval_binary)
numpy_ter=np.array(original_data_eval_ter)
numpy_combo=np.array(original_data_eval_combo)
np.save('./Updated_Modeling/og_binarytest_increasingdata_eval-rows_og_library_columns_ter',numpy_binary,allow_pickle=False)
np.save('./Updated_Modeling/og_tertest_increasingdata_eval-rows_og_library_columns_ter',numpy_ter,allow_pickle=False)
np.save('./Updated_Modeling/og_combotest_increasingdata_eval-rows_og_library_columns_ter',numpy_combo,allow_pickle=False)


reduced_data_eval_binary=[] #list of lists - each 'row' or list is a different amount of library data, each value in the row is different amount of terpolymer data
reduced_data_eval_ter=[]
reduced_data_eval_combo=[]
print('reduced')
for i in range(len(reduced_only_training_sets)):
    sublist_binary=[]
    sublist_ter=[]
    sublist_combo=[]
    print('i',i)
    for j in range(len(ter_training_set)):
        print('j',j)
        xset=np.concatenate(reduced_only_training_sets[:i+1]+ter_training_set[:j+1],axis=0)
        ys=np.concatenate(y_reduced_only_sets[:i+1]+y_ter_sets[:j+1],axis=0)
        trained=train_rf(reduced_optim_C,xset,ys,'reduced_data_optim_red_sets_'+str(i)+'_ter_sets_'+str(j))
        eval=[]
        for i in range(len(Xs_red)):
            eval+=[trained.score(Xs_red[i],Ys_red[i])]
        sublist_binary+=[eval[0]]
        sublist_ter+=[eval[1]]
        sublist_combo+=[eval[2]]
    reduced_data_eval_binary+=[sublist_binary]
    reduced_data_eval_ter+=[sublist_ter]
    reduced_data_eval_combo+=[sublist_combo]
numpyred_binary=np.array(reduced_data_eval_binary)
numpyred_ter=np.array(reduced_data_eval_ter)
numpyred_combo=np.array(reduced_data_eval_combo)
np.save('./Updated_Modeling/red_binarytest_increasingdata_eval-rows_red_library_columns_ter',numpyred_binary,allow_pickle=False)
np.save('./Updated_Modeling/red_tertest_increasingdata_eval-rows_red_library_columns_ter',numpyred_ter,allow_pickle=False)
np.save('./Updated_Modeling/red_combotest_increasingdata_eval-rows_red_library_columns_ter',numpyred_combo,allow_pickle=False)



print("original binary",numpy_binary)
print("original ter",numpy_ter)
print("original combo",numpy_combo)
print("reduced binary",numpyred_binary)
print("reduced ter",numpyred_ter)
print("reduced combo",numpyred_combo)

i 0
j 0
0 (109, 109) 0 (48, 109)
X (157, 109)
Y (157,)
j 1
0 (109, 109) 1 (49, 109)
X (206, 109)
Y (206,)
j 2
0 (109, 109) 2 (50, 109)
X (256, 109)
Y (256,)
j 3
0 (109, 109) 3 (55, 109)
X (311, 109)
Y (311,)
j 4
0 (109, 109) 4 (61, 109)
X (372, 109)
Y (372,)
i 1
j 0
1 (109, 109) 0 (48, 109)
X (266, 109)
Y (266,)
j 1
1 (109, 109) 1 (49, 109)
X (315, 109)
Y (315,)
j 2
1 (109, 109) 2 (50, 109)
X (365, 109)
Y (365,)
j 3
1 (109, 109) 3 (55, 109)
X (420, 109)
Y (420,)
j 4
1 (109, 109) 4 (61, 109)
X (481, 109)
Y (481,)
i 2
j 0
2 (109, 109) 0 (48, 109)
X (375, 109)
Y (375,)
j 1
2 (109, 109) 1 (49, 109)
X (424, 109)
Y (424,)
j 2
2 (109, 109) 2 (50, 109)
X (474, 109)
Y (474,)
j 3
2 (109, 109) 3 (55, 109)
X (529, 109)
Y (529,)
j 4
2 (109, 109) 4 (61, 109)
X (590, 109)
Y (590,)
i 3
j 0
3 (109, 109) 0 (48, 109)
X (484, 109)
Y (484,)
j 1
3 (109, 109) 1 (49, 109)
X (533, 109)
Y (533,)
j 2
3 (109, 109) 2 (50, 109)
X (583, 109)
Y (583,)
j 3
3 (109, 109) 3 (55, 109)
X (638, 109)
Y (638,)
j 4
3 (109, 109

In [100]:
i=4
j=0
xset=np.concatenate(reduced_only_training_sets[:i+1]+ter_training_set[:j+1],axis=0)
print(np.shape(xset))
print(y_reduced_only_sets[i])
print(y_ter_sets[:j+1])
ys=np.concatenate(y_reduced_only_sets[:i+1]+y_ter_sets[:j+1],axis=0)
print(np.shape(ys))

(245, 39)
1
[array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1])]


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 4 has 0 dimension(s)

In [108]:
from sklearn.metrics.pairwise import cosine_similarity
#print(original_only_emd_train[0])
original_similarity=cosine_similarity(original_only_emd_train[0])
np.save('./Updated_Modeling/original_only_similarities',original_similarity,allow_pickle=False)
original_ter=cosine_similarity(original_ter_emd_train[0])
np.save('./Updated_Modeling/original_ter_similarities',original_ter,allow_pickle=False)
#print(original_similarity)